In [ ]:
%matplotlib inline

width = 6
height = 3
import matplotlib
matplotlib.rcParams['figure.figsize'] = [width, height]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hmmlearn

from hmmlearn.hmm import GaussianHMM

In [ ]:
print(pd.__version__)
print(np.__version__)
print(hmmlearn.__version__)

## Look at the data

In [ ]:
nile = pd.read_csv("Nile.csv", index_col = 0)

In [ ]:
nile.head()

In [ ]:
plt.plot(nile.year, nile.val)

## Let's take a look at the hmmlearn API

In [ ]:
vals = np.expand_dims(nile.val.values, 1)
n_states = 2
model = GaussianHMM(n_components=n_states, n_iter=100).fit(vals)
hidden_states = model.predict(vals)

In [ ]:
np.bincount(hidden_states)

In [ ]:
plt.plot(hidden_states)

## Exercise: how can we package this more conveniently?

In [ ]:

def fitHMM(vals, n_states):
    vals = np.reshape(vals,[len(vals),1])
    
    # fit Gaussian HMM to Q
    model = GaussianHMM(n_components=n_states, n_iter=100).fit(vals)
     
    # classify each observation as state 0 or 1
    hidden_states = model.predict(vals)
 
    # fit HMM parameters
    mus = np.squeeze(model.means_)
    sigmas = np.squeeze(np.sqrt(model.covars_))
    transmat = np.array(model.transmat_)
    print(mus)
    print(sigmas)
    
#     # re-order parameters in ascending order of mean of underlying distribution
#     idx      = np.argsort(mus)
#     mus      = mus[idx]
#     sigmas   = sigmas[idx]
#     transmat = transmat[idx, :][:, idx]
    
#     state_dict = {}
#     states = [i for i in range(n_states)]
#     for i in idx:
#         state_dict[i] = states[idx[i]]
    
#     relabeled_states = [state_dict[h] for h in hidden_states]
    relabeled_states = hidden_states
    return (relabeled_states, mus, sigmas, transmat, model)

In [ ]:
hidden_states, mus, sigmas, transmat, model = fitHMM(nile.val.values, 2)

## Exercise: how might we be able to plot this more sensibly?

In [ ]:
def plot_states(ts_vals, states, time_vals):
    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('Year)')
    ax1.set_ylabel('Nile river flow',         color=color)
    ax1.plot(time_vals, ts_vals,      color=color)
    ax1.tick_params(axis='y',            labelcolor=color)

    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('Hidden state', color=color)  
    ax2.plot(time_vals,states,     color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  
    plt.show()

In [ ]:
plot_states(nile.val, hidden_states, nile.year)

## Exercise: can we improve on the analysis above?

### Cut off the 'special' region

In [ ]:
np.where(hidden_states == 0)

In [ ]:
hidden_states, mus, sigmas, transmat, model = fitHMM(nile.val.values, 3)


In [ ]:
plot_states(nile.val, hidden_states, nile.year)

In [ ]:
mus

In [ ]:
np.set_printoptions(precision = 3, suppress = True)

In [ ]:
transmat

In [ ]:
mus

## Exercise: generate new synthetic data from the model and then fit it with a fresh HMM model

#### Easy to sample from an existing HMM model

In [ ]:
res = np.squeeze(model.sample(1000)[0])

In [ ]:
plt.plot(res)

#### Then refit

In [ ]:
hidden_states, mus, sigmas, transmat, model = fitHMM(res, 3)

In [ ]:
def plot_states_no_time(ts_vals, states):
    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('Time)')
    ax1.set_ylabel('Value',        color=color)
    ax1.plot(ts_vals,              color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('Hidden state', color=color)  
    ax2.plot(states,        color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  
    plt.show()

In [ ]:
plot_states_no_time(res[1:100], hidden_states[1:100])

In [ ]:
transmat